# Referrers 변환 스크립트

## 목적
- `referrers` 필드를 `{name, empNo, deptCode}` → `{name, emailId}` 형식으로 변환

## 매핑 전략
1. **1차**: empNo → emailId (사원번호로 매핑)
2. **2차**: name → emailId (사원번호 없으면 이름으로 fallback)

In [1]:
# 공백이 추가되게 변환되어서 이 코드는 쓰지 않는걸 추천 /노트++에서 콜론 뒤에 공백 한칸 들어가는거 없애줘야함 /
'''
찾기: ": (["\[\{0-9tfn])
바꾸기: ":$1

JSON 키-값 구분자 콜론에만 공백 추가됨 (문자열 값 내부는 안 바뀜):

": " (문자열 값)
": [ (배열 값)
": { (객체 값)
": 숫자 (숫자 값)
": true, ": false, ": null
'''

import pandas as pd
import json
import os
from collections import defaultdict

## 0. 경로 설정

In [2]:
# ========== 경로 설정 (필요시 수정) ==========
BASE_DIR = r'C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관'

IN_EMPLOYEE_CSV = os.path.join(BASE_DIR, 'in_employee.csv')
OUT_EMPLOYEE_CSV = os.path.join(BASE_DIR, 'out_employee.csv')
CMDS_INPUT_PATH = os.path.join(BASE_DIR, 'documents_all_converted.cmds')
CMDS_OUTPUT_PATH = os.path.join(BASE_DIR, 'documents_referrers_fixed.cmds')
FAILED_REPORT_PATH = os.path.join(BASE_DIR, 'referrers_mapping_failed.txt')

print(f"입력 cmds: {CMDS_INPUT_PATH}")
print(f"출력 cmds: {CMDS_OUTPUT_PATH}")

입력 cmds: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\documents_all_converted.cmds
출력 cmds: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\documents_referrers_fixed.cmds


## 1. 매핑 테이블 생성

In [3]:
# CSV 파일 로드
in_emp = pd.read_csv(IN_EMPLOYEE_CSV, encoding='utf-8-sig')
out_emp = pd.read_csv(OUT_EMPLOYEE_CSV, encoding='utf-8-sig')

print(f"현재 재직자: {len(in_emp)}명")
print(f"퇴직자: {len(out_emp)}명")
print(f"\n컬럼: {list(in_emp.columns)}")

현재 재직자: 167명
퇴직자: 336명

컬럼: ['사원명', 'ID', '부서', '사원번호', '직위', '부서코드']


In [4]:
# 두 CSV 합치기
all_emp = pd.concat([in_emp, out_emp], ignore_index=True)
print(f"전체 직원 수: {len(all_emp)}명")

# 매핑 딕셔너리 생성
# 1차: empNo → emailId
empno_to_email = {}
for _, row in all_emp.iterrows():
    empno = str(row['사원번호']).strip() if pd.notna(row['사원번호']) else ''
    email_id = str(row['ID']).strip() if pd.notna(row['ID']) else ''
    if empno and empno != 'nan' and email_id:
        empno_to_email[empno] = email_id

# 2차: name → emailId (동명이인 없다고 가정)
name_to_email = {}
for _, row in all_emp.iterrows():
    name = str(row['사원명']).strip() if pd.notna(row['사원명']) else ''
    email_id = str(row['ID']).strip() if pd.notna(row['ID']) else ''
    if name and email_id:
        name_to_email[name] = email_id

print(f"empNo 매핑 수: {len(empno_to_email)}개")
print(f"name 매핑 수: {len(name_to_email)}개")

전체 직원 수: 503명
empNo 매핑 수: 131개
name 매핑 수: 503개


In [5]:
# 매핑 샘플 확인
print("=== empNo → emailId 샘플 ===")
for i, (k, v) in enumerate(list(empno_to_email.items())[:5]):
    print(f"  {k} → {v}")

print("\n=== name → emailId 샘플 ===")
for i, (k, v) in enumerate(list(name_to_email.items())[:5]):
    print(f"  {k} → {v}")

=== empNo → emailId 샘플 ===
  201208860817 → hbpark
  201303580207 → jssong
  201506901005 → shshin
  201303840725 → wjshin
  201911940120 → smsim

=== name → emailId 샘플 ===
  외주계약(공통) → contract
  patent → patent
  베트남경제연구소 → vnomics
  세금계산서(Dell) → tax-dell
  화상회의2 → meet100


## 2. cmds 파일 로드 및 파싱

In [6]:
# 파일 로드
with open(CMDS_INPUT_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()

print(f"총 문서 수: {len(lines)}개")
print(f"\n첫 번째 줄 샘플 (앞 200자):")
print(lines[0][:200] + "...")

총 문서 수: 23320개

첫 번째 줄 샘플 (앞 200자):
addDocument {"sourceId":"doc_15167823_06","docNum":"ANY-사업-791","references":[],"docType":"DRAFT","title":"[LG화학 실험실행시스템 구축]에 대한 매출/매입 계약 품의","attaches":[{"name":"C000059472-1 개별계약서 정보시스템개발구축분야.pdf","...


In [7]:
def parse_cmd_line(line):
    """addDocument {...} 형태의 줄에서 JSON 파싱"""
    line = line.strip()
    if not line.startswith('addDocument '):
        return None, line
    
    json_str = line[len('addDocument '):]
    try:
        data = json.loads(json_str)
        return data, None
    except json.JSONDecodeError as e:
        return None, f"JSON 파싱 에러: {e}"

# 파싱 테스트
test_data, test_err = parse_cmd_line(lines[0])
if test_data:
    print("파싱 성공!")
    print(f"sourceId: {test_data.get('sourceId')}")
    print(f"referrers: {test_data.get('referrers')}")
else:
    print(f"파싱 실패: {test_err}")

파싱 성공!
sourceId: doc_15167823_06
referrers: [{'name': '김민서', 'empNo': '201602881005', 'deptCode': 'AB30'}]


## 3. referrers 변환 함수

In [8]:
def convert_referrer(ref, empno_map, name_map):
    """
    단일 referrer 변환
    {name, empNo, deptCode} → {name, emailId}
    
    Returns: (converted_ref, success, match_method)
    """
    name = ref.get('name', '')
    empno = str(ref.get('empNo', '')).strip() if ref.get('empNo') else ''
    
    email_id = None
    match_method = None
    
    # 1차: empNo로 매핑
    if empno and empno in empno_map:
        email_id = empno_map[empno]
        match_method = 'empNo'
    # 2차: name으로 매핑
    elif name and name in name_map:
        email_id = name_map[name]
        match_method = 'name'
    
    if email_id:
        return {'name': name, 'emailId': email_id}, True, match_method
    else:
        # 매핑 실패 - 원본 유지하되 emailId 없음 표시
        return {'name': name, 'emailId': None}, False, None

def convert_referrers(referrers, empno_map, name_map):
    """
    referrers 배열 전체 변환
    
    Returns: (converted_list, stats)
    """
    if not referrers:
        return [], {'total': 0, 'success': 0, 'by_empno': 0, 'by_name': 0, 'failed': []}
    
    converted = []
    stats = {'total': len(referrers), 'success': 0, 'by_empno': 0, 'by_name': 0, 'failed': []}
    
    for ref in referrers:
        conv_ref, success, method = convert_referrer(ref, empno_map, name_map)
        converted.append(conv_ref)
        
        if success:
            stats['success'] += 1
            if method == 'empNo':
                stats['by_empno'] += 1
            else:
                stats['by_name'] += 1
        else:
            stats['failed'].append(ref)
    
    return converted, stats

## 4. 전체 변환 실행

In [9]:
# 전체 통계
total_stats = {
    'docs_total': 0,
    'docs_with_referrers': 0,
    'referrers_total': 0,
    'referrers_success': 0,
    'referrers_by_empno': 0,
    'referrers_by_name': 0,
    'referrers_failed': 0,
    'failed_list': []  # (sourceId, referrer_info)
}

converted_lines = []
parse_errors = []

for i, line in enumerate(lines):
    line = line.strip()
    if not line:
        converted_lines.append('')
        continue
    
    data, err = parse_cmd_line(line)
    if err:
        parse_errors.append((i+1, err))
        converted_lines.append(line)  # 원본 유지
        continue
    
    total_stats['docs_total'] += 1
    
    # referrers 변환
    referrers = data.get('referrers', [])
    if referrers:
        total_stats['docs_with_referrers'] += 1
        converted_refs, stats = convert_referrers(referrers, empno_to_email, name_to_email)
        
        # 통계 업데이트
        total_stats['referrers_total'] += stats['total']
        total_stats['referrers_success'] += stats['success']
        total_stats['referrers_by_empno'] += stats['by_empno']
        total_stats['referrers_by_name'] += stats['by_name']
        total_stats['referrers_failed'] += len(stats['failed'])
        
        for failed_ref in stats['failed']:
            total_stats['failed_list'].append((data.get('sourceId'), failed_ref))
        
        # 변환된 referrers로 교체
        data['referrers'] = converted_refs
    
    # 다시 cmd 형식으로 변환
    converted_line = 'addDocument ' + json.dumps(data, ensure_ascii=False)
    converted_lines.append(converted_line)

print("=== 변환 완료 ===")

=== 변환 완료 ===


## 5. 검증 및 통계

In [10]:
print("=" * 60)
print("변환 통계 리포트")
print("=" * 60)
print(f"\n[문서 통계]")
print(f"  총 문서 수: {total_stats['docs_total']:,}개")
print(f"  referrers 있는 문서: {total_stats['docs_with_referrers']:,}개")

print(f"\n[Referrer 변환 통계]")
print(f"  총 referrer 수: {total_stats['referrers_total']:,}개")
print(f"  변환 성공: {total_stats['referrers_success']:,}개 ({total_stats['referrers_success']/max(total_stats['referrers_total'],1)*100:.1f}%)")
print(f"    - empNo로 매핑: {total_stats['referrers_by_empno']:,}개")
print(f"    - name으로 매핑: {total_stats['referrers_by_name']:,}개")
print(f"  변환 실패: {total_stats['referrers_failed']:,}개")

if parse_errors:
    print(f"\n[파싱 에러]")
    print(f"  파싱 에러 수: {len(parse_errors)}개")
    for line_no, err in parse_errors[:5]:
        print(f"    Line {line_no}: {err}")

변환 통계 리포트

[문서 통계]
  총 문서 수: 23,320개
  referrers 있는 문서: 7,423개

[Referrer 변환 통계]
  총 referrer 수: 12,110개
  변환 성공: 12,110개 (100.0%)
    - empNo로 매핑: 10,818개
    - name으로 매핑: 1,292개
  변환 실패: 0개


In [11]:
# 매핑 실패한 referrer 목록
if total_stats['failed_list']:
    print("=" * 60)
    print("매핑 실패한 Referrer 목록")
    print("=" * 60)
    
    # 실패한 이름들 집계
    failed_names = defaultdict(int)
    for source_id, ref in total_stats['failed_list']:
        name = ref.get('name', 'UNKNOWN')
        failed_names[name] += 1
    
    print(f"\n고유 실패 이름 수: {len(failed_names)}개\n")
    
    # 빈도순 정렬
    sorted_failed = sorted(failed_names.items(), key=lambda x: -x[1])
    for name, count in sorted_failed:
        print(f"  {name}: {count}회")
else:
    print("\n✅ 모든 referrer 매핑 성공!")


✅ 모든 referrer 매핑 성공!


In [12]:
# 샘플 비교 (referrers가 있는 문서 찾아서 before/after 비교)
print("=" * 60)
print("변환 전/후 샘플 비교")
print("=" * 60)

sample_count = 0
for i, orig_line in enumerate(lines):
    orig_line = orig_line.strip()
    if not orig_line.startswith('addDocument '):
        continue
    
    orig_data, _ = parse_cmd_line(orig_line)
    if not orig_data or not orig_data.get('referrers'):
        continue
    
    conv_data, _ = parse_cmd_line(converted_lines[i])
    
    print(f"\n[문서 {i+1}: {orig_data.get('sourceId')}]")
    print(f"  BEFORE: {json.dumps(orig_data.get('referrers'), ensure_ascii=False)}")
    print(f"  AFTER:  {json.dumps(conv_data.get('referrers'), ensure_ascii=False)}")
    
    sample_count += 1
    if sample_count >= 3:
        break

변환 전/후 샘플 비교

[문서 1: doc_15167823_06]
  BEFORE: [{"name": "김민서", "empNo": "201602881005", "deptCode": "AB30"}]
  AFTER:  [{"name": "김민서", "emailId": "hikim"}]

[문서 7: doc_15179751_06]
  BEFORE: [{"name": "김민서", "empNo": "201602881005", "deptCode": "AB30"}, {"name": "최기원", "empNo": "201612670212", "deptCode": "AB30"}]
  AFTER:  [{"name": "김민서", "emailId": "hikim"}, {"name": "최기원", "emailId": "kwchoi67"}]

[문서 21: doc_15213173_06]
  BEFORE: [{"name": "최기원", "empNo": "201612670212", "deptCode": "AB30"}]
  AFTER:  [{"name": "최기원", "emailId": "kwchoi67"}]


In [13]:
# referrer 개수 일치 검증
print("=" * 60)
print("Referrer 개수 일치 검증")
print("=" * 60)

count_mismatch = []
for i, orig_line in enumerate(lines):
    orig_line = orig_line.strip()
    if not orig_line.startswith('addDocument '):
        continue
    
    orig_data, _ = parse_cmd_line(orig_line)
    conv_data, _ = parse_cmd_line(converted_lines[i])
    
    if not orig_data or not conv_data:
        continue
    
    orig_refs = orig_data.get('referrers', [])
    conv_refs = conv_data.get('referrers', [])
    
    if len(orig_refs) != len(conv_refs):
        count_mismatch.append((orig_data.get('sourceId'), len(orig_refs), len(conv_refs)))

if count_mismatch:
    print(f"\n❌ 개수 불일치: {len(count_mismatch)}건")
    for source_id, orig_cnt, conv_cnt in count_mismatch[:10]:
        print(f"  {source_id}: {orig_cnt} → {conv_cnt}")
else:
    print(f"\n✅ 모든 문서의 referrer 개수 일치!")

Referrer 개수 일치 검증

✅ 모든 문서의 referrer 개수 일치!


## 6. 결과 저장

In [14]:
# 결과 파일 저장
with open(CMDS_OUTPUT_PATH, 'w', encoding='utf-8') as f:
    for line in converted_lines:
        f.write(line + '\n')

print(f"✅ 변환된 파일 저장 완료: {CMDS_OUTPUT_PATH}")
print(f"   파일 크기: {os.path.getsize(CMDS_OUTPUT_PATH) / 1024 / 1024:.2f} MB")

✅ 변환된 파일 저장 완료: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\documents_referrers_fixed.cmds
   파일 크기: 895.50 MB


In [15]:
# 매핑 실패 목록도 별도 저장 (확인용)
if total_stats['failed_list']:
    with open(FAILED_REPORT_PATH, 'w', encoding='utf-8') as f:
        f.write("=" * 60 + "\n")
        f.write("매핑 실패한 Referrer 목록\n")
        f.write("=" * 60 + "\n\n")
        
        # 고유 이름 집계
        failed_names = defaultdict(list)
        for source_id, ref in total_stats['failed_list']:
            name = ref.get('name', 'UNKNOWN')
            failed_names[name].append(source_id)
        
        f.write(f"고유 실패 이름 수: {len(failed_names)}개\n\n")
        
        for name, source_ids in sorted(failed_names.items()):
            f.write(f"{name} ({len(source_ids)}회)\n")
            for sid in source_ids[:3]:  # 최대 3개만 예시
                f.write(f"  - {sid}\n")
            if len(source_ids) > 3:
                f.write(f"  ... 외 {len(source_ids)-3}건\n")
            f.write("\n")
    
    print(f"✅ 실패 목록 저장: {FAILED_REPORT_PATH}")
else:
    print("매핑 실패 없음 - 실패 목록 파일 생성 안함")


매핑 실패 없음 - 실패 목록 파일 생성 안함


## 완료!

### 결과 파일
- `documents_referrers_fixed.cmds` - 변환된 cmds 파일
- `referrers_mapping_failed.txt` - 매핑 실패 목록 (있는 경우)